# imports

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import nbconvert
import requests
import time
from splinter import Browser

# create browser path

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\slaye\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


# visit rotten tomatoes website for top 100
# grab table through pd.read_html in browser

In [3]:
url = "https://www.rottentomatoes.com/top/bestofrt/"
browser.visit(url)
html = browser.html
soup = bs(html,'html.parser')
tables = pd.read_html(url)

# grab correct top 100 table from rotten tomatoes website
# turn it into a dataframe

In [4]:
table = tables[2]

In [5]:
pd.set_option('display.max_rows', None)
table

,Rank,RatingTomatometer,Title,No. of Reviews
0,1.0,99%,It Happened One Night (1934),97
1,2.0,98%,Modern Times (1936),108
2,3.0,96%,Black Panther (2018),521
3,4.0,99%,Citizen Kane (1941),117
4,5.0,99%,The Wizard of Oz (1939),145
5,6.0,98%,Parasite (Gisaengchung) (2019),458
6,7.0,94%,Avengers: Endgame (2019),541
7,8.0,99%,Casablanca (1942),122
8,9.0,97%,Knives Out (2019),464
9,10.0,93%,Us (2019),549


# create csv of top 100 rotten tomatoes for later

In [6]:
table.to_csv('RT_ratings.csv')

# create function which will scrape top 100 imdb movies from website
# function uses browser and soupifies it into an html
# then find all grabs and locates all the items in the top 100 list 
# use for loop to find each indivual movie and rating header in list 
# replaces new line html '\n' from rating and title with nothing
# append to list

In [7]:
def imdb_parse(browser,imdb_list):
    html = browser.html
    soup = bs(html, 'html.parser')
    pieces = soup.find_all('div', class_='lister-item')
    for piece in pieces:
        title = (piece.find('h3', class_='lister-item-header').text).replace('\n', '') 
        title=title.split('.',maxsplit=1)[1]
        rating = piece.find('div', class_='ratings-imdb-rating').text.replace('\n', '')
        imdb_list.append([title,rating])

# call function and use find_by_partial_text to click to the next page then call function again 

In [8]:
imdb_list = []
url = 'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc&ref_=adv_prv'
browser.visit(url)
imdb_parse(browser,imdb_list)
browser.links.find_by_partial_text('Next').click()
imdb_parse(browser,imdb_list)


# use for loop to get only the movie title and put it into a list for later 

In [9]:
imdb_movie = []
for item in imdb_list:
    imdb_movie.append(item[0])

# 

# create function which uses google chrome to look up rotten tomatoes {item}
# click by partial text to find the first link which has www.rottentomatoes.com/
# use beautiful soup to find the text of the html to then find scoreboard which contains rotten tomatoes rating 
# call function

In [10]:
imdb_to_rt_list = []
def IMDB_to_RT():
    for item in imdb_movie:
        movie = f'rotten tomatoes {item}'
        url = f'https://www.google.com/search?q={movie}&safe=active&rlz=1C1VDKB_enUS933US933&ei=cZzDYJP3JcyU-gS767qwCA&oq=rotten+tomatoes%3A+the+god+father+1972&gs_lcp=Cgdnd3Mtd2l6EAMyBggAEBYQHjoHCAAQRxCwAzoHCAAQsAMQQzoKCC4QsAMQyAMQQzoCCAA6CAgAELEDEIMBOgUIABCxAzoCCC46BQguEJMCOgQIABANOggIABAWEAoQHjoICAAQCBANEB46BggAEA0QHjoICCEQFhAdEB46BwghEAoQoAFKBQg4EgExUPiGAlivpAJguKYCaAVwAngBgAGeAYgBtBGSAQQyMC41mAEAoAEBqgEHZ3dzLXdpesgBD8ABAQ&sclient=gws-wiz&ved=0ahUKEwjT85LPjJDxAhVMip4KHbu1DoYQ4dUDCA4&uact=5'
        browser.visit(url)
        browser.click_link_by_partial_href('https://www.rottentomatoes.com/')
        soup = bs(html,'html.parser')
        sopa = bs(browser.find_by_css('score-board').first.value)
        sopa2 = sopa.find('body').get_text()
        imdb_to_rt_list.append(sopa2)

In [11]:
IMDB_to_RT()

C:\Users\slaye\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:485: FutureWarning: browser.find_link_by_partial_href is deprecated. Use browser.links.find_by_partial_href instead.
  FutureWarning,


# clean up data by splitting each movie scoreboard into a list with html '\n' as the split
# append to a list

In [12]:
movie_bodies = []
for body in imdb_to_rt_list:
    splitter = body.split('\n')
    movie_bodies.append(splitter)

In [13]:
movie_bodies

[['THE SHAWSHANK REDEMPTION',
  'R 1994, Drama, 2h 22m',
  '91%',
  'TOMATOMETER',
  '77 Reviews',
  '98%',
  'AUDIENCE SCORE',
  '250,000+ Ratings'],
 ['THE GODFATHER',
  'R 1972, Drama/Crime, 2h 57m',
  '97%',
  'TOMATOMETER',
  '132 Reviews',
  '98%',
  'AUDIENCE SCORE',
  '250,000+ Ratings'],
 ['THE DARK KNIGHT',
  'PG-13 2008, Fantasy/Adventure, 2h 32m',
  '94%',
  'TOMATOMETER',
  '344 Reviews',
  '94%',
  'AUDIENCE SCORE',
  '250,000+ Ratings'],
 ['THE GODFATHER, PART II',
  'R 1974, Drama/Crime, 3h 20m',
  '96%',
  'TOMATOMETER',
  '114 Reviews',
  '97%',
  'AUDIENCE SCORE',
  '250,000+ Ratings'],
 ['12 ANGRY MEN',
  '1957, Drama, 1h 35m',
  '100%',
  'TOMATOMETER',
  '55 Reviews',
  '97%',
  'AUDIENCE SCORE',
  '100,000+ Ratings'],
 ['THE LORD OF THE RINGS: THE RETURN OF THE KING',
  'PG-13 2003, Adventure/Fantasy, 3h 20m',
  '93%',
  'TOMATOMETER',
  '275 Reviews',
  '86%',
  'AUDIENCE SCORE',
  '250,000+ Ratings'],
 ['PULP FICTION',
  'R 1994, Drama/Crime, 2h 33m',
  '92%',


# create lists for ratings 

In [14]:
tomatometer = []
audience_score = []
Rating_year_genre_time = []
def imdb_tomate_audience():
    for items in movie_bodies:
        tomatometer.append(items[2])
        audience_score.append(items[5])
        Rating_year_genre_time.append(items[1])

# call function and put tomatometer list into dataframe
# put all relevant imdb and tomatometer data into separate dataframes

In [15]:
imdb_tomate_audience()

In [16]:
tomate_df = pd.DataFrame(tomatometer)
audience_df = pd.DataFrame(audience_score)
info_df = pd.DataFrame(Rating_year_genre_time)
movie = pd.DataFrame(imdb_movie)
imdb_score = pd.DataFrame(imdb_list)

# create one dataframe from above dataframes and load into a csv  

In [17]:
data = [movie[0],tomate_df[0],audience_df[0],imdb_score[1]]
headers = ['Movie','Tomatometer','Audience Score','IMDB score']
imdb_to_rt_final = pd.concat(data, axis=1, keys=headers)

In [18]:
imdb_to_rt_final.to_csv('Imdb_to_rt.csv')

In [ ]:
browser.quit()